![alt text](<화면 캡처 2025-06-02 201331.png>)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
data = pd.read_csv('xy.dat', sep=r'\s+', names=['x','y', 'err_y'])
print(data.shape)
print(data.describe())

(14, 3)
               x          y      err_y
count  14.000000  14.000000  14.000000
mean    6.864286  13.947857   2.005000
std     3.563621   7.761888   0.813783
min     2.000000   4.280000   0.990000
25%     3.875000   6.077500   1.305000
50%     6.750000  13.040000   2.000000
75%     9.275000  20.235000   2.702500
max    12.800000  25.890000   3.220000


In [ ]:
def CS_2d(vars, data):
    a = vars[0]; b = vars[1]
    res = np.sum(((data['y'] - a - b * data['x'])/data['err_y'])**2)
    return res

def downhill(pos, data, dim=2):
    N = dim + 1
    CS_res = np.zeros(N)
    for _ in range(N):
        CS_res[_] = CS_2d(pos[_], data)
    
    
    
    return pos